Ten kod oblicza tzw. **współczynnik hańby** Polskiego Sejmu.

In [ ]:
import os
import logging
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(asctime)s\t%(message)s')

import re
from tqdm import tqdm
from collections import defaultdict

In [ ]:
from typing import Union, Optional, List

In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

In [ ]:
from aipolit.utils.globals import AIPOLIT_10TERM_SEJM_TRANSCRIPTS_DIR
from hipisejm.stenparser.transcript import SessionTranscript, SpeechReaction, SpeechInterruption, SessionSpeech
from hipisejm.stenparser.transcript_utils import leave_only_specific_type_utt

In [ ]:
import aipolit.transcript.occurrence_counter
from aipolit.transcript.utt_sentence_splitter import UttSentenceSplitter

import importlib
importlib.reload(aipolit.transcript.occurrence_counter)

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as pg
import pandas as pd

In [ ]:
TRANSCRIPT_TYPE_TO_PROCESS = 'sejm'

In [ ]:
transcripts = []

processing_dir = os.path.join(AIPOLIT_10TERM_SEJM_TRANSCRIPTS_DIR, TRANSCRIPT_TYPE_TO_PROCESS)
for filename in sorted(os.listdir(processing_dir)):
    if re.match(r"^.*\.xml$", filename):
        filepath = os.path.join(processing_dir, filename)
        transcript = SessionTranscript()
        transcript.load_from_xml(filepath)
        transcripts.append(transcript)
        
print(f"We have loaded {len(transcripts)} transcripts from: {processing_dir}")

In [ ]:
HANBA_TOKENS = [
    'hańba',
    'hańby',
    'hańbie',
    'hańbę',
    'hańbą',
    'hańbie',
    'hańbo',
    'hańb',
    'hańbom',
    'hańbami',
    'hańbach',
    'haniebny',
    'haniebna',     
    'haniebne',
    'haniebnymi',
    'haniebnych',
]

ZDRADA_TOKENS = [
    'zdrada',
    'zdrady',
    'zdradzie',
    'zdradę',
    'zdradą',
    'zdrado',
    'zdrajca',
    'zdrajcy',
    'zdrajcę',
    'zdrajcą',
    'zdrajco',
    'zdrajców',
    'zdrajcom',
    'zdrajcami',
    'zdrajcach',
    'zdradziecki',
    'zdradziecka',
    'zdradzieckie',
    'zdradzieckimi',
    'zdradzieckich',
]

AGENT_TOKENS = [
    'agent',
    'agenta',
    'agentowi',
    'agentem',
    'agencie',
    'agenci',
    'agentów',
    'agentom',
    'agentami',
    'agentach',
    'agentura',
    'agentury',
    'agenturze',
    'agenturę',
    'agenturą',
    'agenturo',
    'agentur',
    'agenturom',
    'agentury',
    'agenturami',
    'agenturach',
]

WSTYD_TOKENS = [
    'wstyd',
    'wstydu',
    'wstydowi',
    'wstydem',
    'wstydzie',
    'wstydów',
    'wstydach',
    'wstydami',
    'wstydy',
]


hanba_counter = aipolit.transcript.occurrence_counter.ListOccurrenceCounter(HANBA_TOKENS)
zdrada_counter = aipolit.transcript.occurrence_counter.ListOccurrenceCounter(ZDRADA_TOKENS)

In [ ]:
hanba_results = []
zdrada_results = []
for transcript in tqdm(transcripts):
    hresult = hanba_counter.run_count(transcript)
    zresult = zdrada_counter.run_count(transcript)
    hanba_results.append(hresult)
    zdrada_results.append(zresult)

In [ ]:
transcripts_dates = []

total_hanbas_count = []
total_zdradas_count = []
for transcript, hanba_occur, zdrada_occur in zip(transcripts, hanba_results, zdrada_results):
    transcripts_dates.append(transcript.session_date)
    total_hanbas_count.append(len(hanba_occur))
    total_zdradas_count.append(len(zdrada_occur))

In [ ]:
def show_plot_chart_with_hanba_zdrada(title, dates, data1, name1, data2, name2):
    counter_data_df = pd.DataFrame({
        'date': dates, 
        'val1': data1, 
        'val2': data2},
    )

    counter_data_df['date'] = counter_data_df['date'].astype(str)
    plot = pg.Figure(data=[
        pg.Bar(
            name = name1,
            x = counter_data_df['date'],
            y = counter_data_df['val1'],
        ),
        pg.Bar(
            name = name2,
            x = counter_data_df['date'],
            y = counter_data_df['val2']
       )
    ])
    plot.update_xaxes(type='category')
        
    plot.update_layout(
        title=title,
        xaxis_title="Data posiedzenia",
        yaxis_title="Liczba wystąpień",
    )         
    plot.show()

In [ ]:
show_plot_chart_with_hanba_zdrada(
    "Liczba wystąpień 'Hańba, zdrada' na posiedzieniach Sejmu X kadencji",
    transcripts_dates,
    total_hanbas_count,
    'Hańba',
    total_zdradas_count,
    'Zdrada',
)

In [ ]:
print(f"HAŃBA: Ogółem wystąpień od początku kadencji: {sum(total_hanbas_count)} na {len(transcripts)} dni posiedzeń.")
print(f"Daje to średnio {sum(total_hanbas_count)/len(total_hanbas_count)} hańby na dzień posiedzenia.")
print()
print(f"ZDRADA: Ogółem wystąpień od początku kadencji: {sum(total_zdradas_count)} na {len(transcripts)} dni posiedzeń.")
print(f"Daje to średnio {sum(total_zdradas_count)/len(total_zdradas_count)} zdrajców na dzień posiedzenia.")

In [ ]:
def count_reactions_only(results):
    reactions = []
    for transcript, occurrences in zip(transcripts, results):
        reactions_count = 0
        for occurrence in occurrences:
            if isinstance(occurrence.utt_ref, SpeechReaction) or  isinstance(occurrence.utt_ref, SpeechInterruption):
                reactions_count += 1
        reactions.append(reactions_count)
    return reactions
        
total_hanbas_reactions_count = count_reactions_only(hanba_results)
total_zdradas_reactions_count = count_reactions_only(zdrada_results)

In [ ]:
show_plot_chart_with_hanba_zdrada(
    "Liczba wystąpień 'Hańba, zdrada' jako reakcja na czyjeś wystąpienie na posiedzieniach Sejmu X kadencji",
    transcripts_dates,
    total_hanbas_reactions_count,
    'Hańba',
    total_zdradas_reactions_count,
    'Zdrada',
)

In [ ]:
print(f"HAŃBA: Ogółem wystąpień jako przerwanie innej wypowiedzi od początku kadencji: {sum(total_hanbas_reactions_count)} na {len(transcripts)} dni posiedzeń.")
print(f"Daje to średnio {sum(total_hanbas_reactions_count)/len(total_hanbas_reactions_count)} przerwanej hańby na dzień posiedzenia.")
print()
print(f"ZDRADA: Ogółem wystąpień jako przerwanie innej wypowiedzi od początku kadencji: {sum(total_zdradas_reactions_count)} na {len(transcripts)} dni posiedzeń.")
print(f"Daje to średnio {sum(total_zdradas_reactions_count)/len(total_zdradas_reactions_count)} przerwanej zdrady na dzień posiedzenia.")

Przodownicy hańby i zdrady
=======================
Sprawdzimy w czyich ustach najczęściej pojawia się słowo hańba i zdrada!

In [ ]:
def count_speaker_to_occurrences(dates, results):
    speaker_to_occurrences = defaultdict(list)

    for date, occur in zip(dates, results):   
        for occurrence in occur:
            speaker_to_occurrences[occurrence.speaker].append((date, occurrence))
    return speaker_to_occurrences

hanba_speaker_to_occurrences = count_speaker_to_occurrences(transcripts_dates, hanba_results)
zdrada_speaker_to_occurrences = count_speaker_to_occurrences(transcripts_dates, zdrada_results)
                                                            

In [ ]:
def create_ranking(speaker_to_occurrences):
    ranking = 0
    raw_data = []
    for speaker, occurrences in speaker_to_occurrences.items():
        dates_set = set([d for d, _ in occurrences])
        raw_data.append((speaker, len(occurrences), len(dates_set)))
    
    ranking_data = []
    for i, entry in enumerate(sorted(raw_data, key=lambda x: (-x[1], -x[2], x[0]))):
        ranking_data.append((i+1, *entry))
    return ranking_data

In [ ]:
hanba_ranking = create_ranking(hanba_speaker_to_occurrences)
zdrada_ranking = create_ranking(zdrada_speaker_to_occurrences)

In [ ]:
def show_ranking(title, ranking_data):       
    data_df = pd.DataFrame(ranking_data, columns=['No.', 'Mówca', 'Liczba wystąpień', "W różnych dniach"])
    pd.set_option('display.max_rows', len(data_df))
    print()
    print(title)
    display(data_df.style.hide())
    pd.reset_option('display.max_rows')

In [ ]:
show_ranking("Ranking osób (nad)używających 'Hańba'", hanba_ranking)

In [ ]:
show_ranking("Ranking osób (nad)używających 'Zdrada'", zdrada_ranking)

Wszystkie wypowiedzenia
======================

Teraz sprawdźmy sobie jakiego rodzaju wypowiedzi zawierają hańba i zdrada.

Uwaga... to jest dużo tekstu :)

Dla przejrzystości wypowiedzi powtarzające się zostały scalone w jedno (i poprzedzone liczbą wystąpień). Kiedy wypowiedź jest unikalna wyświetlimy też sobie datę, aby można było sprawdzić szerszy kontekst.

In [ ]:
def show_all_sentences(ranking_data, speaker_to_occurrences):
    raw_data = []
    for ranking_entry in ranking_data:
        ranking = ranking_entry[0]
        speaker = ranking_entry[1]
        occurrences = speaker_to_occurrences[speaker]
        sentence_to_dates = defaultdict(list)
        for occur in occurrences:
            date = occur[0]
            phrase_occur = occur[1]
            sentence = phrase_occur.sentence
            if isinstance(phrase_occur.utt_ref, SpeechInterruption):
                sentence = f"[przerywa] {sentence}"
                
            sentence_to_dates[sentence].append(date)
        sentences_sorted = []
        for sentence, dates in sorted(sentence_to_dates.items(), key = lambda k_v: (-len(k_v[1]), k_v[1][0])):
            if 1 == len(dates):
                sentences_sorted.append(f"({dates[0]}) {sentence}")
            else:
                sentences_sorted.append(f"[x{len(dates)}] ({dates[0]}, ...) {sentence}")
        sentences_sorted_merged = "<br>".join(sentences_sorted)
        raw_data.append((ranking, speaker, sentences_sorted_merged))
    
    data_df = pd.DataFrame(raw_data, columns=['No.', 'Mówca', 'Wystąpienia'])
    pd.set_option('display.max_rows', len(data_df))
    df_styler = data_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
    display(df_styler.hide())
    pd.reset_option('display.max_rows')

Wszystkie wypowiedzenia z 'Hańbą'
=================================

In [ ]:
show_all_sentences(hanba_ranking, hanba_speaker_to_occurrences)

Wszystkie wypowiedzenia ze 'Zdradą'
=================================

In [ ]:
show_all_sentences(zdrada_ranking, zdrada_speaker_to_occurrences)

Komu się najczęściej przerywa?
==============================

W tej części sprawdzimy czyje wystąpienia są najczęściej przerywane okrzykami hańba i zdrada.

In [ ]:
def count_speaker_to_interruptions(dates, results):
    speaker_to_interruptions = defaultdict(list)
    for date, occurrences in zip(dates, results):
        for occurrence in occurrences:
            if isinstance(occurrence.utt_ref, SpeechInterruption):
                interrupted_speaker = occurrence.speech_ref.speaker
                speaker_to_interruptions[interrupted_speaker].append((date, occurrence))
    return speaker_to_interruptions

hanba_speaker_to_interruptions = count_speaker_to_interruptions(transcripts_dates, hanba_results)
zdrada_speaker_to_interruptions = count_speaker_to_interruptions(transcripts_dates, zdrada_results)

In [ ]:
def cut_long(sent, length=100):
    if len(sent) <= length:
        return sent
    else:
        return sent[0:length] + ' [...]'

def show_ranking_of_interruptions(speaker_to_interruptions):
    raw_data = []
    ranking = 0
    for speaker, interruptions in sorted(speaker_to_interruptions.items(), key= lambda k_v: (-len(k_v[1]), k_v[0])):
        ranking += 1
        
        who_interrupted = []
        for entry in interruptions:
            date = entry[0]
            occurrence = entry[1]            
            who_interrupted.append(
                f"({date}) {occurrence.speaker}: {occurrence.sentence} [po zdaniu] {cut_long(occurrence.prev_sentence, 150)}")
            
        who_interrupted_merged = "<br>".join(who_interrupted)
        
        raw_data.append((ranking, speaker, len(interruptions), who_interrupted_merged))
        
    data_df = pd.DataFrame(raw_data, columns=['No.', 'Mówca, któremu przerwano', 'Przerwań', 'Kto przerywa'])
    pd.set_option('display.max_rows', len(data_df))
    df_styler = data_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
    display(df_styler.hide())
    pd.reset_option('display.max_rows')

Ranking komu najczęściej przerywa się słowami 'Hańba!'
==============================================

In [ ]:
show_ranking_of_interruptions(hanba_speaker_to_interruptions)

Ranking komu najczęściej przerywa się słowami 'Zdrada'
==============================================

In [ ]:
show_ranking_of_interruptions(zdrada_speaker_to_interruptions)

Czy przerywane wypowiedzi coś łączy?
===================================

Materiału powyżej nie ma zbyt dużo, jednakże spróbujemy i tak sprawdzić czy da się znaleźć jakiekolwiek punkty wspólne.

W tym celu proponuję następujące podejście:

1. Zbierzemy N zdań (nie tylko jedno, aby złapać szerszy kontekst) wypowiedzenia, które zostało przerwane (jeżeli wypowiedzenie jest krótsze, to oczywiście weźmiemy całość wypowiedzi do momentu przerwania).
2. Z każdego takiego fragmentu spróbujemy wyciągnąć słowa kluczowe, które następnie posłużą nam do scharakteryzowania każdego wypowiedzenia.
3. Sprawdzimy frekwencję pozyskanych słów kluczowych - być może niektóre występują statystycznie częściej
4. Dodatkowo dokonamy analizy czy pewne słowa kluczowe nie są do siebie podobne - w tym miejscu wykorzystamy analizę skupień (clustering) i odpowiednio zaktualizujemy ranking. 

W drugim mniej ambitnym podejściu zapytamy o zdanie LLMa ;)

In [ ]:
utt_sentence_splitter = UttSentenceSplitter()

In [ ]:
MAX_INTERRUPTED_FRAGMENT_LENGTH = 7 # number of sentences!

def process_utt_for_split(result: List[str], utt: str, max_length: int) -> bool:
    splitted_list = utt_sentence_splitter.split_utt_to_sentences(utt)
    for sentence in reversed(splitted_list):
        result.append(sentence)
        if len(result) >= max_length:
            return True
    return False

def retrieve_interrupted_fragment(occurrence, max_length) -> List[str]:
    """
    Retrieces only "norm" utts (without interruptions/reactions) from the given speech
    to the place where occurrence sentence appears.
    Occurrence is not taken into account to the result if it is reaction or interruption!
    max_length - max number of sentences to retrieve!
    """
    occurrence_sentence = occurrence.sentence
    speech_ref = occurrence.speech_ref
    matched_utt_index = occurrence.matched_utt_index
    
    utts_until_matched = speech_ref.content[:matched_utt_index]
    utts_strings_only = leave_only_specific_type_utt(utts_until_matched, str)
    
    result = []
    # now we will split utts into sentences, to leave only max_length
    for utt in reversed(utts_strings_only):
        if process_utt_for_split(result, utt, max_length):
            break
    return list(reversed(result))


def create_interrupted_fragments(results):
    interrupted_fragments = []
    for occurrences in results:
        for occurrence in occurrences:
            if isinstance(occurrence.utt_ref, SpeechInterruption):
                interrupted_speaker = occurrence.speech_ref.speaker
                
                interrupted_fragment = retrieve_interrupted_fragment(occurrence, max_length=MAX_INTERRUPTED_FRAGMENT_LENGTH)
                entry = (interrupted_fragment, occurrence)
                interrupted_fragments.append(entry)
                                
    return interrupted_fragments
    
hanba_interrupted_fragments = create_interrupted_fragments(hanba_results)

Sprawdźmy jeszcze tylko jak dużo tekstu udało nam się wyciągnąć do analizy...

In [ ]:
def quick_estimate_of_fragments_size(fragments):
    stats = {
        'sentences': 0,
        'tokens': 0,
        'characters': 0,
    }
    
    for frag in fragments:
        stats['sentences'] += len(frag[0])
        for sentence in frag[0]:
            tokens = re.split(r"\s+", sentence)
            stats['tokens'] += len(tokens)
            stats['characters'] += len(sentence)
        
    print(f"Całkowita liczba zdań: {stats['sentences']}" )
    print(f"Całkowita liczba wyrazów (tokenów): {stats['tokens']}" )
    print(f"Całkowita liczba znaków: {stats['characters']}" )
    
quick_estimate_of_fragments_size(hanba_interrupted_fragments)

Aby pozyskać słowa kluczowe wykorzystamy moduł: https://github.com/MaartenGr/KeyBERT

Każdy z fragmentów zostanie potraktowany jako dokument, z którego pozyskamy słowa kluczowe.

In [ ]:
sample_doc = " ".join(hanba_interrupted_fragments[6][0])
print("Sample doc:\n\n" + sample_doc)

In [ ]:
# source: https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt
pl_stopwords = """a
aby
ach
acz
aczkolwiek
aj
albo
ale
alez
ależ
ani
az
aż
bardziej
bardzo
beda
bedzie
bez
deda
będą
bede
będę
będzie
bo
bowiem
by
byc
być
byl
byla
byli
bylo
byly
był
była
było
były
bynajmniej
cala
cali
caly
cała
cały
ci
cie
ciebie
cię
co
cokolwiek
cos
coś
czasami
czasem
czemu
czy
czyli
daleko
dla
dlaczego
dlatego
do
dobrze
dokad
dokąd
dosc
dość
duzo
dużo
dwa
dwaj
dwie
dwoje
dzis
dzisiaj
dziś
gdy
gdyby
gdyz
gdyż
gdzie
gdziekolwiek
gdzies
gdzieś
go
i
ich
ile
im
inna
inne
inny
innych
iz
iż
ja
jak
jakas
jakaś
jakby
jaki
jakichs
jakichś
jakie
jakis
jakiś
jakiz
jakiż
jakkolwiek
jako
jakos
jakoś
ją
je
jeden
jedna
jednak
jednakze
jednakże
jedno
jego
jej
jemu
jesli
jest
jestem
jeszcze
jeśli
jezeli
jeżeli
juz
już
kazdy
każdy
kiedy
kilka
kims
kimś
kto
ktokolwiek
ktora
ktore
ktorego
ktorej
ktory
ktorych
ktorym
ktorzy
ktos
ktoś
która
które
którego
której
który
których
którym
którzy
ku
lat
lecz
lub
ma
mają
mało
mam
mi
miedzy
między
mimo
mna
mną
mnie
moga
mogą
moi
moim
moj
moja
moje
moze
mozliwe
mozna
może
możliwe
można
mój
mu
musi
my
na
nad
nam
nami
nas
nasi
nasz
nasza
nasze
naszego
naszych
natomiast
natychmiast
nawet
nia
nią
nic
nich
nie
niech
niego
niej
niemu
nigdy
nim
nimi
niz
niż
no
o
obok
od
około
on
ona
one
oni
ono
oraz
oto
owszem
pan
pana
pani
po
pod
podczas
pomimo
ponad
poniewaz
ponieważ
powinien
powinna
powinni
powinno
poza
prawie
przeciez
przecież
przed
przede
przedtem
przez
przy
roku
rowniez
również
sam
sama
są
sie
się
skad
skąd
soba
sobą
sobie
sposob
sposób
swoje
ta
tak
taka
taki
takie
takze
także
tam
te
tego
tej
ten
teraz
też
to
toba
tobą
tobie
totez
toteż
totobą
trzeba
tu
tutaj
twoi
twoim
twoj
twoja
twoje
twój
twym
ty
tych
tylko
tym
u
w
wam
wami
was
wasz
wasza
wasze
we
według
wiele
wielu
więc
więcej
wlasnie
właśnie
wszyscy
wszystkich
wszystkie
wszystkim
wszystko
wtedy
wy
z
za
zaden
zadna
zadne
zadnych
zapewne
zawsze
ze
zeby
zeznowu
zł
znow
znowu
znów
zostal
został
żaden
żadna
żadne
żadnych
że
żeby""".split("\n")

In [ ]:
# TODO disabled

from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings
from collections import Counter

# wybierzemy model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 
# rekomendowany przez KeyBert
# można też sprawdzić modele z benchmarku https://klejbenchmark.com/leaderboard/
kw_model_name = 'allegro/herbert-large-cased'
kw_model_name = 'sdadas/polish-roberta-large-v2'
kw_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

kw_model_embedding = TransformerDocumentEmbeddings(kw_model_name)
kw_model = KeyBERT(model=kw_model_embedding)

all_keywords = Counter()
for i, hanba_frag in tqdm(enumerate(hanba_interrupted_fragments)):
    # TODO this does not work as good as ChatGPT :(
    break
    hanba_doc = " ".join(hanba_frag[0])          
    
    keywords = kw_model.extract_keywords(
        hanba_doc, 
        keyphrase_ngram_range=(1, 3), 
        stop_words=pl_stopwords, 
        highlight=False, 
        use_maxsum=True, 
        nr_candidates=10, 
        top_n=5, 
        use_mmr=True, 
        diversity=0.9)
    for kw in keywords:
        all_keywords[kw[0]] += 1

rank = 0
for kw, freq in sorted(all_keywords.items(), key=lambda k_v: -k_v[1]):
    rank += 1
    
    print(f"{rank}. {kw} {freq}")
    if rank > 20:
        break

In [ ]:
documents_for_keybert = []
for frag in hanba_interrupted_fragments:
    sents = " ".join(frag[0])
    documents_for_keybert.append(sents)

In [ ]:
from aipolit.models.keybert_llm_cached import KeyBertLLMCached
kw_model = KeyBertLLMCached()

keywords = kw_model.extract_keywords(documents_for_keybert)

In [ ]:
all_keywords = Counter()
for doc, keys in zip(documents_for_keybert, keywords):
    
    print("DOCUMENT:", doc)
    print("Tematy:", keys)
    for kw in keys:
        all_keywords[kw] += 1
    print("")

In [ ]:
rank = 0
for kw, freq in sorted(all_keywords.items(), key=lambda k_v: -k_v[1]):
    rank += 1
    
    print(f"{rank}. {kw} {freq}")

No a teraz chcielibyśmy pogrupować te wszystkie tematy, bo niektóre z nich są bardzo podobne (np. Donald Tusk, premier Donald Tusk, premier Tusk). Do tego wykorzystamy analizę skupień. Ponieważ nie wiemy jaka liczba grup jest optymalna użyjemy algorytmu DBScan, który pozwala łączyć elementy w grupy tak długo, aż nie zostanie przekroczona pewna wartość graniczna. Hiperparametry alogrytmu dobrałem metodą prób i błędów :)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

def cluster_topics(topics, show_plot=True):    
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    embeddings = model.encode(topics)

    # hyperparamters are chosen after some experiments
    dbscan = DBSCAN(eps=0.15, min_samples=2, metric='cosine')
    clusters = dbscan.fit_predict(embeddings)

    if show_plot:
        tsne = TSNE(n_components=2, random_state=42)
        embeddings_2d = tsne.fit_transform(embeddings)

        plt.figure(figsize=(10, 7))
        unique_labels = set(clusters)
        colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))

        for label, color in zip(unique_labels, colors):
            if label == -1:
                color = 'gray'
                marker = 'x'
            else:
                marker = 'o'
    
            plt.scatter(
                embeddings_2d[clusters == label, 0],
                embeddings_2d[clusters == label, 1],
                c=[color],
                marker=marker,
                label=f'Klaster {label}' if label != -1 else 'Szum'
            )

        plt.title('Analiza skupień tematów (DBSCAN)')
        plt.show()

    cluster_to_topics = defaultdict(list)
    for cluster_id, topic in zip(clusters, topics):
        cluster_to_topics[cluster_id].append(topic)
    return cluster_to_topics


In [ ]:
to_cluster = [k for k in all_keywords.keys() if k != 'brak tematów']

cluster_to_topics = cluster_topics(to_cluster)

Zobaczymy jakiego rodzaju grupy powstały w wyniku działania alogrytmu.

In [ ]:
def print_clustered_topics(cluster_to_topics):
    rank = 0
    for cls_id, topics in sorted(cluster_to_topics.items(), key=lambda k_v: -len(k_v[1])):
        if cls_id < 0:
            continue
        rank += 1
        print(f"{rank}. Liczba tematów: {len(topics)}. Tematy: {', '.join(topics)}")


print_clustered_topics(cluster_to_topics)         

Pozostaje teraz policzyć frekwencję wystąpień tematów w ramach każdej z grup. To da nam ranking tematów, które pojawiają się najczęściej przed reakcją "Hańba!". No i trzeba teraz rozbić sztuczny klaster -1 na osobne grupy.

In [ ]:
cluster_to_freq_and_topics = dict()

# na początek rozbijemy klaster -1 na pojedyncze grupy
for topic in cluster_to_topics[-1]:
    new_id = -(len(cluster_to_freq_and_topics) + 1)
    cluster_to_freq_and_topics[new_id] = ([topic], all_keywords[topic])

# a teraz dodajmy większe klastry
for cluster_id, topics in cluster_to_topics.items():
    if cluster_id < 0:
        continue
    freq = 0
    for topic in topics:
        freq += all_keywords[topic]
    cluster_to_freq_and_topics[cluster_id] = (topics, freq)

rank = 0
for cluster_id, entry in sorted(cluster_to_freq_and_topics.items(), key=lambda k_v: -k_v[1][1]):
    rank += 1
    print(f"{rank}. Wystąpień: {entry[1]}. Liczba tematów: {len(entry[0])}. Tematy: {', '.join(entry[0])}")